**Deep learning with 2 different dataset and ensemble**

In [9]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
import joblib
import pickle

# Load data
df=pd.read_excel('EOGsignal.xlsx').iloc[:, 1:]
# Rename the columns
new_column_names = ['f' + str(i) for i in range(0, 24)] + ['class']
df.columns = new_column_names

eog_data = df
emg_data = pd.read_csv('EMGF1.csv').values
y_data = np.load('y_test_combined.npy')

# Balance the datasets
n_samples = min(len(eog_data), len(emg_data), len(y_data))

eog_data_balanced = resample(eog_data, n_samples=n_samples, random_state=42)
emg_data_balanced = resample(emg_data, n_samples=n_samples, random_state=42)
y_data_balanced = resample(y_data, n_samples=n_samples, random_state=42)

# Split balanced data into training and testing sets
X_train_eog, X_test_eog, y_train, y_test = train_test_split(eog_data_balanced, y_data_balanced, test_size=0.2, random_state=42)
X_train_emg, X_test_emg, y_train_emg, y_test_emg = train_test_split(emg_data_balanced, y_data_balanced, test_size=0.2, random_state=42)

# Convert y_train and y_test for EOG data to one-hot encoded labels with 5 classes
y_train_eog = to_categorical(np.clip(y_train - 1, 0, 4), num_classes=5)
y_test_eog = to_categorical(np.clip(y_test - 1, 0, 4), num_classes=5)

# Convert y_train and y_test for EMG data to one-hot encoded labels with 7 classes
y_train_emg = to_categorical(y_train_emg, num_classes=7)
y_test_emg = to_categorical(y_test_emg, num_classes=7)

import tensorflow as tf
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

def build_nn_model(input_shape, num_classes):
    model = Sequential()
    
    # Input layer
    model.add(Dense(256, activation='relu', input_shape=(input_shape,)))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    
    # Hidden layers
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    
    # Output layer
    model.add(Dense(num_classes, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
    
    return model

# Create model instances for EOG and EMG data
eog_model = build_nn_model(X_train_eog.shape[1], num_classes=5)
emg_model = build_nn_model(X_train_emg.shape[1], num_classes=7)

# Define early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train EOG model
eog_model.fit(X_train_eog, y_train_eog, epochs=100, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

# Train EMG model
emg_model.fit(X_train_emg, y_train_emg, epochs=100, batch_size=32, validation_split=0.2, callbacks=[early_stopping])


# Save models
eog_model.save('eog_nn_model.h5')
emg_model.save('emg_nn_model.h5')


Epoch 1/100
70/70 [==============================] - 1s 3ms/step - loss: 1.7427 - accuracy: 0.3190 - val_loss: 1.0175 - val_accuracy: 0.7292
Epoch 2/100
70/70 [==============================] - 0s 1ms/step - loss: 1.2528 - accuracy: 0.6046 - val_loss: 1.0048 - val_accuracy: 0.7292
Epoch 3/100
70/70 [==============================] - 0s 1ms/step - loss: 1.1100 - accuracy: 0.6823 - val_loss: 0.9845 - val_accuracy: 0.7292
Epoch 4/100
70/70 [==============================] - 0s 2ms/step - loss: 1.0644 - accuracy: 0.7022 - val_loss: 0.9576 - val_accuracy: 0.7292
Epoch 5/100
70/70 [==============================] - 0s 1ms/step - loss: 1.0548 - accuracy: 0.7045 - val_loss: 0.9492 - val_accuracy: 0.7292
Epoch 6/100
70/70 [==============================] - 0s 1ms/step - loss: 1.0362 - accuracy: 0.7122 - val_loss: 0.9740 - val_accuracy: 0.7292
Epoch 7/100
70/70 [==============================] - 0s 2ms/step - loss: 1.0287 - accuracy: 0.7094 - val_loss: 0.9867 - val_accuracy: 0.7292
Epoch 8/100
7

In [10]:

# Load trained models
eog_model = tf.keras.models.load_model('eog_nn_model.h5')
emg_model = tf.keras.models.load_model('emg_nn_model.h5')

# Generate predictions
eog_predictions = eog_model.predict(X_test_eog)
emg_predictions = emg_model.predict(X_test_emg)

# Convert predictions to class labels
eog_predictions = np.argmax(eog_predictions, axis=1) + 1  # Add 1 to match original class labels
emg_predictions = np.argmax(emg_predictions, axis=1)

# Stack predictions
meta_features = np.column_stack((eog_predictions, emg_predictions))

# Ensure y_test is one-hot encoded if necessary
if len(y_test.shape) == 1:
    y_test = to_categorical(y_test)

# Convert y_test to class labels for meta-classifier
y_test_meta = np.argmax(y_test, axis=1)

# Train meta-classifier
meta_classifier = RandomForestClassifier(random_state=42)
meta_classifier.fit(meta_features, y_test_meta)

# Save meta-classifier
with open('meta_classifier_rf.sav', 'wb') as file:
    pickle.dump(meta_classifier, file)

# Predict using meta-classifier
final_predictions = meta_classifier.predict(meta_features)

# Evaluate the performance
accuracy = accuracy_score(y_test_meta, final_predictions)
print(f'Combined Model Accuracy: {accuracy:.2f}')

22/22 [==============================] - 0s 744us/step
Combined Model Accuracy: 0.67
